In [2]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import SGDRegressor, Ridge
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error

import warnings
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_json('clear_data.txt')
df.head()

,room_number,floor,max_floor,general_area,living_area,kitchen_area,price,district
0,1,3,5,30.0,18.0,6.0,1780000,Ленинский
1,1,3,5,31.4,17.0,7.0,2100000,Правобережный
2,1,3,5,31.4,18.0,6.0,1830000,Правобережный
3,1,12,14,36.0,0.0,9.0,1610000,Правобережный
4,1,1,5,32.0,25.0,7.0,1750000,Ленинский


# Подготовка данных

In [4]:
# Векторизируем названия районов
district_vector = pd.get_dummies(df['district'])
prepare_df = pd.concat([df, district_vector], axis=1).drop(columns=['district'])
prepare_df.head()

,room_number,floor,max_floor,general_area,living_area,kitchen_area,price,Ленинский,Орджоникидзевский,Правобережный
0,1,3,5,30.0,18.0,6.0,1780000,1,0,0
1,1,3,5,31.4,17.0,7.0,2100000,0,0,1
2,1,3,5,31.4,18.0,6.0,1830000,0,0,1
3,1,12,14,36.0,0.0,9.0,1610000,0,0,1
4,1,1,5,32.0,25.0,7.0,1750000,1,0,0


In [5]:
# Создаем столбец, который будет отвечать за разность между этажом и максимальным этажом
prepare_df['floor_to_max'] = prepare_df['max_floor'] - prepare_df['floor']
prepare_df.head()

,room_number,floor,max_floor,general_area,living_area,kitchen_area,price,Ленинский,Орджоникидзевский,Правобережный,floor_to_max
0,1,3,5,30.0,18.0,6.0,1780000,1,0,0,2
1,1,3,5,31.4,17.0,7.0,2100000,0,0,1,2
2,1,3,5,31.4,18.0,6.0,1830000,0,0,1,2
3,1,12,14,36.0,0.0,9.0,1610000,0,0,1,2
4,1,1,5,32.0,25.0,7.0,1750000,1,0,0,4


In [7]:
# Разделяем данные на матрицу признаков и таргеты
X = prepare_df[['room_number', 'floor', 'general_area', 'living_area', 'kitchen_area', 'Ленинский', 'Орджоникидзевский', 'Правобережный', 'floor_to_max']]
y = prepare_df['price']

In [8]:
# Разделяем данные на обучающие и тестовые
x_train, x_test, y_train, y_test = train_test_split(X, y)

In [9]:
transformer = StandardScaler()
transformer.fit(x_train)
x_train_trans = transformer.transform(x_train)
x_test_trans = transformer.transform(x_test)

# Выбор модели

Выбирать будем между понятными и эффективными моделями: Random Forest, Gradient Boosting, Ridge, SGDRegressor

Пока гиперпараметры подбирать не будем, сделаем это позже

## Выбор лучшего типа модели

In [10]:
models = {
    'Random Forest': RandomForestRegressor(),
    'Gradient Boosting': GradientBoostingRegressor(),
    'Ridge': Ridge(),
    'SGD': SGDRegressor()
}

In [12]:
metrics_df = pd.DataFrame(columns=['model_name', 'RMSE', 'MAE', 'MAPE'])

for i, model_name in enumerate(models):
    item = {
        'model_name': model_name
    }
    models[model_name].fit(x_train_trans, y_train)
    y_predict = models[model_name].predict(x_test_trans)
    item['RMSE'] = mean_squared_error(y_test, y_predict) ** 0.5
    item['MAE'] = mean_absolute_error(y_test, y_predict)
    item['MAPE'] = mean_absolute_percentage_error(y_test, y_predict)
    item_df = pd.DataFrame([item], index=[i])
    metrics_df = pd.concat([metrics_df, item_df])

metrics_df

,model_name,RMSE,MAE,MAPE
0,Random Forest,488698.416314,259525.049069,0.118906
1,Gradient Boosting,527419.906545,327466.851904,0.150090
2,Ridge,587626.232136,401609.049702,0.183637
3,SGD,587397.191648,400659.372648,0.183715


## Подбор гиперпараметров

In [13]:
param_grid = {
    'n_estimators': list(range(100, 201, 50)),
    'max_depth': list(range(10, 51, 10)),
    'min_samples_split': list(range(2, 11, 3)),
    'min_samples_leaf': list(range(2, 11, 3)),
}

model = RandomForestRegressor()
grid_search = GridSearchCV(estimator=model, param_grid=param_grid)
grid_search.fit(x_train_trans, y_train)

GridSearchCV(estimator=RandomForestRegressor(),
             param_grid={'max_depth': [10, 20, 30, 40, 50],
                         'min_samples_leaf': [2, 5, 8],
                         'min_samples_split': [2, 5, 8],
                         'n_estimators': [100, 150, 200]})

In [14]:
# Обучаем модель с лучшими параметрами и смотрим на качество
model = RandomForestRegressor(**grid_search.best_params_)
model.fit(x_train_trans, y_train)
y_predict = model.predict(x_test_trans)
print(f'RMSE: {mean_squared_error(y_test, y_predict) ** 0.5}')
print(f'MAE: {mean_absolute_error(y_test, y_predict)}')
print(f'MAPE: {mean_absolute_percentage_error(y_test, y_predict)}')

RMSE: 485519.77906405285
MAE: 269566.99913819757
MAPE: 0.12909272677362632


**Промежуточный вывод:** видно, что качество от перебора параметров не улучшилось, поэтому будем использовать базовые настройки модели